In [1]:
## ----Importing necessary stuff----
import os  # Handling filesystems
import warnings
 
# Ignoring a silly warning from HVSRPY
warnings.simplefilter("ignore", UserWarning) 
warnings.simplefilter("ignore", RuntimeWarning) 

import numpy as np  # Numerical array management
import matplotlib.pyplot as plt  # Plot results
from matplotlib.collections import LineCollection
from matplotlib import spines
from tqdm import tqdm  # To track progress of HVSR computations
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import hann
from scipy.integrate import romb, cumulative_simpson, simpson

plt.rcParams['date.converter'] = 'concise'
plt.style.use('seaborn-v0_8-paper')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = 'Arial'  # switch to Helvetica if running on mac

# from obspy.clients.fdsn.client import Client  # Locally stored data
# from obspy.clients.filesystem.sds import Client
from obspy import read, read_inventory, UTCDateTime  